In [10]:
# Imports you may need
from dataloader import *
from finance import *
import pandas as pd

from finance import stock, compare
from quotebankexploration import *
from wikipedia import *

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
# Get some data frames
wiki_data = concat_wiki_files()
filtered_quotes = get_filtered_quotes()
speakers_id = pd.DataFrame(get_speakers_ids(filtered_quotes))
speakers_labels = get_speakers_labels()

In [12]:
yolo = filtered_quotes.copy()
yolo['year'] = filtered_quotes.date.apply(lambda date: date.year)
display(yolo)

,quoteID,quotation,speaker,qids,date,numOccurrences,probas,urls,phase,year
0,2008-10-15-012147,five times the graphics performance.,steve jobs,[Q19837],2008-10-15 06:38:46,2,"[[steve jobs, 0.7707], [None, 0.2293]]",[http://us.rd.yahoo.com/dailynews/rss/search/m...,A,2008
1,2008-09-25-038377,my iphone is full of pictures. i wanted to see...,ken whisenhunt,[Q1758635],2008-09-25 08:08:08,1,"[[ken whisenhunt, 0.3759], [None, 0.3186], [ma...",[http://azcentral.com/arizonarepublic/sports/a...,A,2008
2,2008-09-04-052789,the people who connect needs and ideas the bes...,steve jobs,[Q19837],2008-09-04 18:19:47,1,"[[steve jobs, 0.717], [None, 0.283]]",[http://businessweek.com/magazine/content/07_4...,A,2008
3,2008-11-10-038937,sesame workshop will get 70 percent of the rev...,robert macmillan,"[Q21453558, Q28094302]",2008-11-10 22:59:44,1,"[[robert macmillan, 0.6071], [None, 0.3929]]",[http://macdailynews.com/index.php/weblog/comm...,A,2008
4,2008-09-26-048683,the huge success of the apple itunes app store...,steve howard,"[Q23670647, Q2457386, Q7612886]",2008-09-26 11:29:23,1,"[[steve howard, 0.5034], [None, 0.4966]]",[http://ipod.consumerelectronicsnet.com/articl...,A,2008
...,...,...,...,...,...,...,...,...,...,...
310811,2020-03-06-004716,Apple continues to show their tremendous suppo...,none,[],2020-03-06 01:14:31,2,"[[None, 0.6748], [Octavia Spencer, 0.2615], [M...",[https://www.denofgeek.com/tv/truth-be-told-se...,E,2020
310812,2020-01-21-010221,"But at the same time, we're also not saying th...",satya nadella,[Q7426870],2020-01-21 15:44:24,1,"[[Satya Nadella, 0.8487], [None, 0.1513]]",[https://www.theverge.com/2020/1/21/21071108/m...,E,2020
310813,2020-01-22-079970,The Big Mac burger sauce is so hard to get hol...,none,[],2020-01-22 12:34:13,2,"[[None, 0.8268], [Big Mac, 0.1733]]",[https://www.mirror.co.uk/money/shopping-deals...,E,2020
310814,2020-04-01-004824,Archie has been loving doing FaceTime playdate...,none,[],2020-04-01 12:14:50,1,"[[None, 0.6707], [Prince Harry, 0.2197], [Megh...",[https://www.pinkvilla.com/entertainment/holly...,E,2020


In [ ]:
year = 2015

test = speakers_labels.head(10)

test[str(year)] = test.label.apply(lambda label: get_page_views_per_year(label, year))

In [ ]:
display(test)
test.to_pickle(path)

In [ ]:
print(get_page_views_per_year('Tim Cook', 2015))

In [25]:
quotes_pageviews = filtered_quotes.iloc(0, 1).set_index('speaker').join(test.iloc(0, 1).set_index('speaker'))
display(quotes_pageviews)

TypeError: __call__() takes from 1 to 2 positional arguments but 3 were given

In [ ]:
year = 2015

path = os.path.join('data/wiki_speaker_attributes/pageviews_'+str(year))

print(path)

In [26]:
test = speakers_labels.head(10)
for year in range(2015, 2021):
    test[str(year)] = test.label.apply(lambda label: get_page_views_per_year(label, year))

In [27]:
display(test)

,speaker,qids,label,2015,2016,2017,2018,2019,2020
0,steve jobs,[Q19837],Steve Jobs,6365607.0,9536391.0,7605088.0,6458359.0,6362619.0,6900472.0
1,ken whisenhunt,[Q1758635],Ken Whisenhunt,135492.0,115367.0,86274.0,132915.0,113473.0,81236.0
2,robert macmillan,"[Q21453558, Q28094302]",Robert MacMillan,498.0,1111.0,946.0,884.0,835.0,1152.0
3,steve howard,"[Q23670647, Q2457386, Q7612886]",Steve Howard,9700.0,23643.0,19157.0,17095.0,18213.0,21539.0
4,doug martin,"[Q18685889, Q27995830, Q3037945, Q3714598, Q53...",Doug Martin,3276.0,6785.0,3220.0,6103.0,1841.0,1242.0
5,eddy cue,[Q224628],Eddy Cue,30460.0,55411.0,56605.0,46063.0,46296.0,49487.0
6,tom harman,[Q7816093],Tom Harman,1829.0,3421.0,3195.0,2903.0,2732.0,2704.0
7,none,[],None,NaN,NaN,NaN,NaN,NaN,NaN
8,trevor beattie,"[Q25190178, Q7839055]",Trevor Beattie,2640.0,6212.0,5948.0,5279.0,5290.0,4192.0
9,michael phelps,[Q39562],Michael Phelps,713332.0,11937527.0,1474663.0,1530111.0,1403024.0,1600845.0
